In [ ]:
import av
import numpy as np
import pandas as pd
import os
import cv2

In [ ]:
def check_for_vfr(video_path, tolerance=1e-6):
    """
    Analyzes a video file to determine if it has a Variable Frame Rate.

    Args:
        video_path (str): Path to the video file.
        tolerance (float): The allowed deviation for frame durations.

    Returns:
        bool: True if the video is VFR, False if it is CFR.
    """
    try:
        with av.open(video_path) as container:
            stream = container.streams.video[0]
            
            time_base = stream.time_base
            
            # --- FIX IS HERE ---
            # Convert the Fraction timestamp to a float immediately after calculation.
            frame_timestamps = [float(frame.pts * time_base) for frame in container.decode(stream)]
            
            if len(frame_timestamps) < 2:
                print("Not enough frames to determine VFR.")
                return False

            deltas = np.diff(frame_timestamps)
            
            if np.std(deltas) < tolerance:
                print(f"Video appears to be CFR. Average FPS: {1 / np.mean(deltas):.2f}")
                return False
            else:
                print(f"Video appears to be VFR. FPS standard deviation: {np.std(deltas):.4f}")
                return True

    except Exception as e:
        print(f"An error occurred: {e}")
        return False
        
def frame_iterator_pyav(video_path):
    """
    A memory-efficient iterator that yields frames and their true timestamps as floats.
    This handles both CFR and VFR video correctly.
    """
    with av.open(video_path) as container:
        stream = container.streams.video[0]
        time_base = stream.time_base
        for frame in container.decode(stream):
            # --- FIX IS HERE ---
            # Yield the timestamp as a float.
            timestamp_sec = float(frame.pts * time_base)
            yield frame.to_ndarray(format='bgr24'), timestamp_sec


In [ ]:
!pwd

In [ ]:
for filename in os.listdir('raw_videos'):
    check_for_vfr(f"raw_videos/{filename}")


In [ ]:
csv_list = [pd.read_csv(os.path.join('annotations', csv_path)) for  csv_path in os.listdir('annotations') if 'csv' in csv_path]


In [ ]:
prop_ct = 0
num_csv = len(csv_list)
for s in csv_list:
    length = s['end_time'].iloc[-1]
    #print(length)
    time_in_point = np.sum(s['end_time'] - s['start_time'])
    proportion = time_in_point/length
    prop_ct += proportion
print(prop_ct/num_csv)


# checking current yolo inference files to make sure 15 fps

In [ ]:
# first, single file

np_data = np.load('pose_data/raw/yolos_0.25conf_15fps_0s_to_99999s/Aditi Narayan ｜ Matchplay_posedata_0s_to_99999s_yolos.npz', allow_pickle=True)
video_path = 'raw_videos/Aditi Narayan ｜ Matchplay.mp4'

In [ ]:
for pose_data_path in os.listdir('pose_data/raw/yolos_0.25conf_15fps_0s_to_99999s'):
    if 'npz' not in pose_data_path:
        continue
    np_data = np.load(f'pose_data/raw/yolos_0.25conf_15fps_0s_to_99999s/{pose_data_path}', allow_pickle=True)
    video_path = f'raw_videos/{pose_data_path.split("_posedata")[0]}.mp4'

    frames = np_data['frames']
    not_annotated_count = sum(1 for frame in frames if frame.get('annotation_status') == -100)
    annotated_count = sum(1 for frame in frames if frame.get('annotation_status') != -100)
    fps = get_fps_opencv(video_path)


    print(f"total frames: {not_annotated_count+annotated_count}")
    print(f"annotated frame count: {annotated_count}")
    print(f"video: {video_path.split('/')[-1]}")
    print(f"proportion: {annotated_count/(annotated_count+not_annotated_count)}")
    print(f"fps: {fps}, annotation proportion: {15/fps}")
    print()

In [ ]:
annotated_count/(annotated_count+not_annotated_count)

In [ ]:
15/24

In [ ]:
def get_fps_opencv(video_path):
    """
    Gets the FPS of a video file using OpenCV.

    Args:
        video_path (str): The full path to the video file.

    Returns:
        float: The frames per second (FPS) of the video, or -1 if an error occurs.
    """
    # Check if the file exists to avoid errors
    if not os.path.exists(video_path):
        print(f"Error: Video file not found at {video_path}")
        return -1

    # Create a VideoCapture object
    video_capture = cv2.VideoCapture(video_path)

    # Check if the video was opened successfully
    if not video_capture.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return -1

    # Get the FPS using the CAP_PROP_FPS property
    # For older OpenCV versions, you might use: cv2.cv.CV_CAP_PROP_FPS
    fps = video_capture.get(cv2.CAP_PROP_FPS)

    # Release the VideoCapture object to free up resources
    video_capture.release()

    return fps

In [ ]:
fps = get_fps_opencv(video_path)
fps

# new annotation check

In [ ]:
24741 / 98961

In [ ]:
video_feature_path = 'pose_data/features/yolos_0.25conf_15fps_0s_to_99999s/Aditi Narayan ｜ Matchplay_features.npz'
feature_data = np.load(video_feature_path, allow_pickle=True)

In [ ]:
feature_data.files
